In [1]:
from neo4j import GraphDatabase

URI_container = "neo4j://neo4j:7687"
URI_local = "bolt://localhost:7687"
URI = URI_container

In [ ]:
with GraphDatabase.driver(URI) as driver:
    result = driver.execute_query("""
    match (n:Person)-[r:USED]-(c:CreditCard), (c)-[s:SENT]-(t:Transaction), (t)-[]-(m:Merchant)
    where t.amt > 200
    return t
    """)
    frequent_high_transactions = driver.execute_query("""
    match (c:CreditCard)-[s:SENT]-(t1:Transaction), (c)-[:SENT]-(t2:Transaction)
    where t1.amt > 200 and t2.amt > 200 and t2.trans_date - duration('P7D')<= t1.trans_date <= t2.trans_date + duration('P7D')
    return t1, t2
    order by t1.trans_date
    """)
    duplicate_transaction_different_cards = drive.execute_query("""
    match (c2:CreditCard)-[:SENT]-(t:Transaction)-[:SENT]-(c1:CreditCard)
    where c1.cc_num <> c2.cc_num
    return t,c1,c2
    """)
    frequent_transactions_grocery_pos= drive.execute_query("""
    match (m:Merchant {category:"grocery_pos"})-[:RECEIVED]-(t1:Transaction)-[:SENT]-(c:CreditCard), (c)-[:SENT]-(t2:Transaction)
    where t1.trans_date - duration('P7D') <= t2.trans_date <= t1.trans_date + duration('P7D') and t1.amt > 50 and t2.amt > 50
    return m,t1,t2, c
    """)
    frequent_transactions_online_shopping= drive.execute_query("""
    match (m:Merchant {category:"shopping_net"})-[:RECEIVED]-(t1:Transaction)-[:SENT]-(c:CreditCard), (c)-[:SENT]-(t2:Transaction)
    where t1.trans_date - duration('P7D') <= t2.trans_date <= t1.trans_date + duration('P7D') and t1.amt > 50 and t2.amt > 50
    return m,t1,t2, c
    """)
    transaction_distance = drive.execute_query("""
    match (p:Person)-[:RESIDES_IN]-(l:Location),(p)-[:USED]-(c:CreditCard),(c)-[:SENT]-(t:Transaction),(t)-[:RECEIVED]-(m:Merchant)
    where m.category =~ '.*(?<!net)$'
    with
        point({longitude:l.long, latitude: l.lat, crs: 'WGS-84'}) AS personLoc,
        point({longitude: m.merch_long, latitude: m.merch_lat, crs: 'WGS-84'}) AS merchLoc,
        t.trans_num AS transaction_num,
        m.category AS category
    RETURN transaction_num, category, round(point.distance(personLoc, merchLoc)/1000) AS transactionDistance
    ORDER BY transactionDistance DESC
    limit 1000
    """)
    